In [61]:
# 모듈, 경로 설정
import pandas as pd
import pydeck as pdk
import geopandas as gpda
import config
import json
import requests
import sys
import folium

path = "C:\github/aide-analysis/"
sys.path.append(path)

from src.sql import DatabaseHandler
db = DatabaseHandler()

from core.config import Config
config = Config()


In [194]:
# Vworld 시군구 경계 API 호출
def gl(sig_cd):
    # 엔드포인트
    endpoint = "http://api.vworld.kr/req/data"

    # API 파라미터
    service = "data"
    request = "GetFeature"

    # 행정동 경계
    data = "LT_C_ADSIGG_INFO"

    # 요청 파라미터
    key = config.OPEN_API["vworld"]
    page = 1
    size = 1000
    
    attrFilter = f"sig_cd:like:{11}"
    url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"

    # 요청 결과
    res = json.loads(requests.get(url).text)

    # GeoJson 생성
    featureCollection = {"type": "FeatureCollection"}
    features = []
    while True:
        # 요청 URL
        url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"
        # 요청 결과
        res = json.loads(requests.get(url).text)
        # GeoJson 추가
        features = features + res["response"]["result"]["featureCollection"]["features"]
        # 페이지
        page_current = int(res["response"]["page"]["current"])
        page_total = int(res["response"]["page"]["total"])
        page += 1
        if page_current == page_total:
            break
    featureCollection["features"] = features
    geo = featureCollection
    return geo
geo = gl(11)

In [ ]:
# params
center = [37.541, 126.986]
lon = 127.0080061098534
lat = 37.50989980338221
icon = ['star','flag']
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']


# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 12,
    width = 750,
    height = 500,
    tiles = tiles[2]
)


# Marker
"""
location : 마커를 추가할 경위도 좌표/ 
popup : 표기할 팝업문구(클릭시 문구)/ 
tooltip : 표기할 툴팁 지정(마우스 오버))
"""
folium.Marker(
    location = [lat, lon],
    popup = '반포자이',
    tooltip = '반포자이',
    icon = folium.Icon('red', icon = icon[0])
).add_to(m)

# GeoJson 활용
folium.GeoJson(
    geo,
    name = '서초구'
).add_to(m)

m
# m.save('data/result.html')

In [ ]:
# m.save('result/서울시군구경계시각화.html')

### DB연결후 적용

#### 세대수

In [ ]:
# 데이터 불러오기
sql = f"""
SELECT hdongName, 
       totalSaedae 
FROM aide.population_saedae
WHERE sidoCode ='11' 
      and sigunguCode != '11000' 
      and date = (SELECT MAX(date) 
                  FROM aide.population_saedae);
"""
df = db.read_table(sql)

In [ ]:

# params
center = [37.541, 126.986]
lon = 127.0080061098534
lat = 37.50989980338221
icon = ['star','flag']
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']


# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 12,
    tiles = tiles[2]
)

# Marker
folium.Marker(
    location = [37.5146321, 127.018293],
    popup = '회사',
    tooltip = '회사',
    icon = folium.Icon('red', icon = icon[0])
).add_to(m)


# Marker
folium.Marker(
    location = [37.4857983, 126.948162],
    popup = '집',
    tooltip = '집',
    icon = folium.Icon('red', icon = icon[0])
).add_to(m)


# Polygon
m.choropleth(
    geo_data = geo,
    data = df,
    columns = ('hdongName','totalSaedae'),
    key_on = 'feature.properties.full_nm',
    fill_color = 'YlOrRd',
    fill_opacity = 0.5,
    line_opacity = 0.2
)
m

In [ ]:
# m.save('result/서울시시군구세대수시각화.html')

In [ ]:
location_data = [[37.5146321, 127.018293],
                [37.4857983, 126.948162]]
folium.PolyLine(locations = location_data, tooltip = "Polyline").add_to(m)
m

### 연속지적도 + 아파트 전세 공급평단가(최근날짜기준)

In [4]:
sql = f"""
SELECT T1.pnuCode,
	   T1.contractYear,
       T1.contractMonth,
       T1.tradingSupplyUnitPriceAverage,
       T1.jeonseSupplyUnitPriceAverage,
       T1.wolseSupplyUnitPriceAverage,
       T1.rn
FROM(
SELECT pnuCode, 
	   contractYear, 
	   contractMonth,
       MAX(tradingSupplyUnitPriceAverage) as tradingSupplyUnitPriceAverage,
       MAX(jeonseSupplyUnitPriceAverage) AS jeonseSupplyUnitPriceAverage,
       MAX(wolseSupplyUnitPriceAverage) AS wolseSupplyUnitPriceAverage,
       row_number() OVER(PARTITION BY pnuCode ORDER BY contractYear DESC) AS rn
       
FROM aide.apartment_danji_monthly_price
WHERE pnuCode LIKE '41463%'
GROUP BY pnuCode, contractYear, contractMonth 
) AS T1
WHERE T1.rn=  1
"""
price_df = db.read_table(sql)

In [5]:
price_df = price_df.dropna(subset = 'jeonseSupplyUnitPriceAverage')
price_df = price_df[['pnuCode','contractYear','contractMonth','jeonseSupplyUnitPriceAverage']]
price_df

,pnuCode,contractYear,contractMonth,jeonseSupplyUnitPriceAverage
0,4146310100100020001,2022,1,1308.515162
1,4146310100100060009,2022,3,565.813037
2,4146310100100140000,2022,1,915.155935
3,4146310100100140004,2022,2,1091.504994
4,4146310100100170015,2022,1,1142.190593
...,...,...,...,...
259,4146311800112910000,2022,2,1200.735724
260,4146311800112930000,2022,1,973.119756
261,4146311800112980000,2022,1,913.683445
262,4146311800113930000,2022,1,1043.598671


In [6]:
# join
lst = ",".join(price_df['pnuCode'])

In [7]:
def continue_land():
    # 엔드포인트
    endpoint = "http://api.vworld.kr/req/data"

    # API 파라미터
    service = "data"
    request = "GetFeature"

    # 연속지적도
    data = "LP_PA_CBND_BUBUN"

    # 요청 파라미터
    key = config.OPEN_API["vworld"]
    page = 1
    size = 1000
    
    attrFilter = f'pnu:IN:{lst}'
    url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"

    # 요청 결과
    res = json.loads(requests.get(url).text)

    # GeoJson 생성
    featureCollection = {"type": "FeatureCollection"}
    features = []
    while True:
        # 요청 URL
        url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"
        # 요청 결과
        res = json.loads(requests.get(url).text)
        # GeoJson 추가
        features = features + res["response"]["result"]["featureCollection"]["features"]
        # 페이지
        page_current = int(res["response"]["page"]["current"])
        page_total = int(res["response"]["page"]["total"])
        page += 1
        if page_current== page_total:
            break 
    featureCollection["features"] = features
    geo = featureCollection
    return geo

geo = continue_land()

In [10]:
# geo

In [11]:

# params
center = [37.541, 126.986]
lon = 127.0080061098534
lat = 37.50989980338221
icon = ['star','flag']
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']


# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 12,
    tiles = tiles[0]
)


# Polygon
m.choropleth(
    geo_data = geo,
    data = price_df,
    columns = ('pnuCode','jeonseSupplyUnitPriceAverage'),
    key_on = 'feature.properties.pnu',
    fill_color = 'YlOrRd',
    fill_opacity = 0.5,
    line_opacity = 0.2
)
m

C:\Users\jeongseokLim\anaconda3\envs\aide\lib\site-packages\folium\folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [ ]:
m.save('result/용인시기흥구.html')

### 시군구경계 + 미분양 시각화

#### sig_cd 로 API를 호출하게 되면 시군구 뎁스가 맞지않아 모든 시군구가 매핑이 되지않고, full_nm으로 API를 호출하면 "JSONDecodeError: Expecting value: line 1 column 1 (char 0)" 에러가 발생한다. 일시 중단

In [ ]:
sidoCode

In [12]:
sql = f"""

SELECT * 
FROM aide.unsold_sigungu
WHERE year = '2022' 
      AND month = '03'
      AND sigunguName != '계'
      
"""
df = db.read_table(sql)
df['full_nm'] = df['sidoName']+' '+df['sigunguName']
df

,sidoCode,sigunguCode,sidoName,sigunguName,date,year,month,unsold,full_nm
0,11,11110,서울특별시,종로구,2022-03-01,2022,3,0,서울특별시 종로구
1,11,11140,서울특별시,중구,2022-03-01,2022,3,2,서울특별시 중구
2,11,11170,서울특별시,용산구,2022-03-01,2022,3,0,서울특별시 용산구
3,11,11200,서울특별시,성동구,2022-03-01,2022,3,0,서울특별시 성동구
4,11,11215,서울특별시,광진구,2022-03-01,2022,3,3,서울특별시 광진구
...,...,...,...,...,...,...,...,...,...
223,48,48870,경상남도,함양군,2022-03-01,2022,3,0,경상남도 함양군
224,48,48880,경상남도,거창군,2022-03-01,2022,3,3,경상남도 거창군
225,48,48890,경상남도,합천군,2022-03-01,2022,3,0,경상남도 합천군
226,50,50110,제주특별자치도,제주시,2022-03-01,2022,3,389,제주특별자치도 제주시


In [13]:
lst = ','.join(df['full_nm'])

In [14]:
lst

'서울특별시 종로구,서울특별시 중구,서울특별시 용산구,서울특별시 성동구,서울특별시 광진구,서울특별시 동대문구,서울특별시 중랑구,서울특별시 성북구,서울특별시 강북구,서울특별시 도봉구,서울특별시 노원구,서울특별시 은평구,서울특별시 서대문구,서울특별시 마포구,서울특별시 양천구,서울특별시 강서구,서울특별시 구로구,서울특별시 금천구,서울특별시 영등포구,서울특별시 동작구,서울특별시 관악구,서울특별시 서초구,서울특별시 강남구,서울특별시 송파구,서울특별시 강동구,부산광역시 중구,부산광역시 서구,부산광역시 동구,부산광역시 영도구,부산광역시 부산진구,부산광역시 동래구,부산광역시 남구,부산광역시 북구,부산광역시 해운대구,부산광역시 사하구,부산광역시 금정구,부산광역시 강서구,부산광역시 연제구,부산광역시 수영구,부산광역시 사상구,부산광역시 기장군,대구광역시 중구,대구광역시 동구,대구광역시 서구,대구광역시 남구,대구광역시 북구,대구광역시 수성구,대구광역시 달서구,대구광역시 달성군,인천광역시 중구,인천광역시 남동구,인천광역시 서구,인천광역시 연수구,인천광역시 미추홀구,인천광역시 동구,인천광역시 부평구,인천광역시 계양구,인천광역시 강화군,인천광역시 옹진군,광주광역시 동구,광주광역시 서구,광주광역시 남구,광주광역시 북구,광주광역시 광산구,대전광역시 동구,대전광역시 중구,대전광역시 서구,대전광역시 유성구,대전광역시 대덕구,울산광역시 중구,울산광역시 남구,울산광역시 동구,울산광역시 북구,울산광역시 울주군,경기도 수원시,경기도 성남시,경기도 의정부시,경기도 안양시,경기도 부천시,경기도 광명시,경기도 평택시,경기도 동두천시,경기도 안산시,경기도 고양시,경기도 과천시,경기도 구리시,경기도 남양주시,경기도 오산시,경기도 시흥시,경기도 군포시,경기도 의왕시,경기도 하남시,경기도 용인시,경기도 파주시,경기도 이천시,경기도 안성시,경기도 김포시,경기도 양주시,경기도 여주시,경기도 화성시,경기도 광주시,경기도 연천군,경기도 포천시,경기도 가평군,경기도 양평군,강원도 춘천시,강원도 원주시,강원도 강릉시,강원도

In [15]:
# 엔드포인트
endpoint = "http://api.vworld.kr/req/data"

# API 파라미터
service = "data"
request = "GetFeature"

# 행정동 경계
data = "LT_C_ADSIGG_INFO"

# 요청 파라미터
key = config.OPEN_API["vworld"]
page = 1
size = 1000

attrFilter = f"full_nm:IN:서울특별시 서초구,서울특별시 종로구"
url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"


json.loads(requests.get(url).text)

{'response': {'service': {'name': 'data',
   'version': '2.0',
   'operation': 'GetFeature',
   'time': '22(ms)'},
  'status': 'OK',
  'record': {'total': '2', 'current': '2'},
  'page': {'total': '1', 'current': '1', 'size': '1000'},
  'result': {'featureCollection': {'type': 'FeatureCollection',
    'bbox': [126.94889856321107,
     37.42829747687726,
     127.09575295335512,
     37.632374766651424],
    'features': [{'type': 'Feature',
      'geometry': {'type': 'MultiPolygon',
       'coordinates': [[[[127.00864326221883, 37.58046825113493],
          [127.00871274905404, 37.58045116421941],
          [127.00876564011087, 37.58044310616639],
          [127.00890785297045, 37.58042423069648],
          [127.00913781377906, 37.58039352848355],
          [127.00916523299747, 37.580390715633335],
          [127.00923792440939, 37.58038254796624],
          [127.00926870590294, 37.58037916725067],
          [127.0092779097711, 37.580378031403306],
          [127.0094381678049, 37.58035

In [26]:
# 엔드포인트
endpoint = "http://api.vworld.kr/req/data"

# API 파라미터
service = "data"
request = "GetFeature"

# 행정동 경계
data = "LT_C_ADSIGG_INFO"

# 요청 파라미터
key = config.OPEN_API["vworld"]
page = 1
size = 1000

attrFilter = f"full_nm:IN:서울특별시종로구"
url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"
print(url)
# res = xmltodict.parse(requests.get(url).text)
# res = json.loads(json.dumps(res))


http://api.vworld.kr/req/data?service=data&request=GetFeature&data=LT_C_ADSIGG_INFO&key=634F43BA-BA1F-33EE-809E-95FF93DD76F1&attrFilter=full_nm:=:서울특별시종로구&page=1&size=1000


In [21]:
# Vworld 시군구 경계 API 호출
import xmltodict
def gl():
    # 엔드포인트
    endpoint = "http://api.vworld.kr/req/data"

    # API 파라미터
    service = "data"
    request = "GetFeature"

    # 행정동 경계
    data = "LT_C_ADSIGG_INFO"

    # 요청 파라미터
    key = config.OPEN_API["vworld"]
    page = 1
    size = 1000
    
    attrFilter = f"full_nm:IN:{lst}"
    url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"


    # GeoJson 생성
    featureCollection = {"type": "FeatureCollection"}
    features = []
    while True:
        # 요청 URL
        url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"
        # 요청 결과
        res = json.dumps(json.loads(requests.get(url).text),ensure_ascii=False)
        
        # res = xmltodict.parse(requests.get(url).text)
        # res = json.loads(json.dumps(res))
        
        # GeoJson 추가
        features = features + res["response"]["result"]["featureCollection"]["features"]
        # 페이지
        page_current = int(res["response"]["page"]["current"])
        page_total = int(res["response"]["page"]["total"])
        page += 1
        if page_current == page_total:
            break
    featureCollection["features"] = features
    geo = featureCollection
    return geo
geo = gl()

ExpatError: syntax error: line 1, column 49

In [20]:
#geo

In [ ]:
# params
center = [37.541, 126.986]
lon = 127.0080061098534
lat = 37.50989980338221
icon = ['star','flag']
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']


# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 12,
    tiles = tiles[0]
)


# Polygon
m.choropleth(
    geo_data = geo,
    data = df,
    columns = ('sigunguCode','unsold'),
    key_on = 'feature.properties.sig_cd',
    fill_color = 'YlGnBu',
    fill_opacity = 0.5,
    line_opacity = 0.2
)
m

### 입력한 지번 반경에 있는 비슷한 스펙의 매물을 조회 및 건물 지하철등 상업시설조회 

**구현하고자 하는 기능:**
1. ~~지번을 입력받으면 해당 지번 물건의 반경 안에 있는 매물들을 모두 지도상에 표시한다~~
2. ~~매물들의 세부정보 popup에 추가~~
3. 해당 지번은 과 비슷한 스펙의 주변 매물들이 지도상에 표시가 된다.
4. ~~마우스를 올리면 매물에 대한 정보가 요약된다. ~~
5. ~~주변 지하철이 표시가 된다.~~
6. 주변 공원 및 큰 상점들이 표시가 된다.

In [30]:
add_list[1]

'경기도 용인시 기흥구 보정로 87'

In [34]:
sigunguCode

'41463'

In [4]:
# 경로설정
import sys
path = "C:\github/aide-analysis/"
sys.path.append(path)

# 모듈 import
import pandas as pd
import src
import folium
from src.openapi import KakaoLocalAPI as kakao
from openpyxl import load_workbook
from src.utils import insert_dataframe_to_excel_sheet
from src.sql import DatabaseHandler
from src.temp import *


db = DatabaseHandler()

# 카카오 API 호출
docs = kakao.search_address('잠원동 22-10')


# 파싱정보
# 지번 추출
jibeon = docs['documents'][0]['address_name']

# 시도/ 시군구/ 행정동 코드 추출
sidoCode = docs["documents"][0]["address"]["h_code"][:2]
sigunguCode = docs["documents"][0]["address"]["h_code"][:5]
hdongCode = docs["documents"][0]["address"]["h_code"]

# 행정구역 명칭
sido_ = docs["documents"][0]["address"]["region_1depth_name"]
sigungu_ = docs["documents"][0]["address"]["region_2depth_name"]
if sigungu_ == "":
    sigungu_ = sido_
hdong_ = docs['documents'][0]['address']['region_3depth_h_name']
if hdong_ == "":
    hdong_ = docs['documents'][0]['address']['region_3depth_name']

# 경도, 위도
lon = docs['documents'][0]['address']['x'] 
lat = docs['documents'][0]['address']['y'] 
radius = 1000

# 단지명
danjiName = docs['documents'][0]['road_address']['building_name']

# 시계열 인자
startyear = datetime.now().year-10
endyear = datetime.now().year-1



In [65]:
# # 가격테이블 JOIN전
# # 단지정보 테이블 
# sql = f"""
# SELECT *, ST_Distance_Sphere(POINT({lon}, {lat}), geom) AS distance
# FROM aide.apartment_danji_information
# WHERE ST_Distance_Sphere(POINT({lon}, {lat}), geom) <= {radius}
# ORDER BY distance ASC
# ;
# """
# things = db.read_table(sql)

# # 반경내 지하철역 정보 테이블
# sql2 = f"""
# SELECT * , 
#        ST_Distance_Sphere(POINT({lon},{lat}),POINT(lon,lat)) AS distance
# FROM aide.railway_station_info
# WHERE ST_Distance_Sphere(POINT({lon},{lat}),POINT(lon,lat)) <= {radius}
# ORDER BY distance ASC;
# """
# train = db.read_table(sql2)

In [172]:
# 입력된지번의 일정 반경내 아파트와 아파트 정보
sql = f"""
WITH tmp AS (SELECT A.pnuCode, addressJibeon, addressRoad, danjiName, usePermissionDate, B.contractYear, B.contractMonth, B.jeonyongArea, B.jeonseJeonyongUnitPriceAverage, buildingAreaPerLandArea, floorAreaRatio, mainPurposeCodeName, otherPurpose, saedaeCount, totalParkingLotCount, lon, lat, coord, geom
            FROM aide.apartment_danji_information AS A
            JOIN (SELECT T1.*
                FROM(
                    SELECT pnuCode,
                            contractYear,
                            contractMonth,
                            jeonyongArea,
                            supplyArea,
                            tradingJeonyongUnitPriceAverage,
                            jeonseJeonyongUnitPriceAverage,
                            wolseJeonyongUnitPriceAverage,
                            ROW_NUMBER() OVER(PARTITION BY pnuCode ,jeonyongArea ORDER BY contractYear DESC, contractMonth DESC) AS num
                    FROM aide.apartment_danji_monthly_area_price
                    WHERE pnuCode LIKE '{sidoCode}%' AND jeonseJeonyongUnitPriceAverage is not null
                    ) AS T1
                    WHERE T1.num = 1 
            ) AS B
            on A.pnuCode = B.pnuCode 
)SELECT tmp.*,
        ST_Distance_Sphere(POINT({lon},{lat}),geom) AS distance
        
FROM tmp
WHERE ST_Distance_Sphere(POINT({lon},{lat}), geom) <={radius}
ORDER BY distance ASC;"""
df= db.read_table(sql)


# 반경내 지하철역 정보 테이블
sql2 = f"""
SELECT * , 
       ST_Distance_Sphere(POINT({lon},{lat}),POINT(lon,lat)) AS distance
FROM aide.railway_station_info
WHERE ST_Distance_Sphere(POINT({lon},{lat}),POINT(lon,lat)) <= {radius}
ORDER BY distance ASC;
"""
train = db.read_table(sql2)

In [173]:
#         iframe = folium.IFrame('지번주소:' + str(df.iloc[i]['addressJibeon']) + '<br>' 
#                        '단지이름: ' + str(df.iloc[i]['danjiName']) + '<br>' 
#                        '사용승인일: ' +str(df.iloc[i]['usePermissionDate']) + '<br>' 
#                        '전용면적: ' +str(df.iloc[i]['jeonyongArea']) + '<br>' 
#                        '전세평균전용평단가: ' +str(df.iloc[i]['jeonseJeonyongUnitPriceAverage']) + '<br>' 
#                       '건폐율: ' +str(df.iloc[i]['buildingAreaPerLandArea']) + '<br>' 
#                       '용적률: ' +str(df.iloc[i]['floorAreaRatio']) + '<br>' 
#                       '주용도코드명: ' +str(df.iloc[i]['mainPurposeCodeName']) + '<br>'
#                       '기타용도: ' +str(df.iloc[i]['otherPurpose']) + '<br>'
#                       '세대수: ' +str(df.iloc[i]['saedaeCount']) + '<br>'
#                       '총주차대수: ' +str(df.iloc[i]['totalParkingLotCount']) + '<br>' 
#                       '거리: ' +str(df.iloc[i]['distance']) + '<br>'),
#         popup = folium.Popup(iframe, min_width=300, max_width=300),
        

In [174]:
from IPython.display import IFrame
icon = ['star','flag']
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']
center = [df.iloc[0]['lat'], df.iloc[0]['lon']]
# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 14,
    width = 750,
    height = 500,
    tiles = tiles[2]
)




# 입력받은 지번을 제외한 매물 마킹
for i in range(1,len(df)):
    folium.Marker(
        location = [df.iloc[i]['lat'], df.iloc[i]['lon']],      
        popup = [df.iloc[i]['addressJibeon'], 
                 df.iloc[i]['danjiName'],
                 df.iloc[i]['usePermissionDate'],
                 df.iloc[i]['jeonyongArea'],
                 df.iloc[i]['jeonseJeonyongUnitPriceAverage'],
                 df.iloc[i]['buildingAreaPerLandArea'],
                 df.iloc[i]['floorAreaRatio'],
                 df.iloc[i]['mainPurposeCodeName'],
                 df.iloc[i]['otherPurpose'],
                 df.iloc[i]['saedaeCount'],
                 df.iloc[i]['totalParkingLotCount'],
                 df.iloc[i]['distance']],
        tooltip = df.iloc[i]['danjiName'],
        icon = folium.Icon('blue', icon = icon[1])).add_to(m)
    
# 입력받은 지번 매물 마킹
folium.Marker(
    location = [lat,lon],
    popup = [f"지번 : {df.iloc[i]['addressJibeon']}", 
         f"단지명 : {df.iloc[0]['danjiName']}",
         df.iloc[0]['usePermissionDate'],
         df.iloc[0]['jeonyongArea'],
         df.iloc[0]['jeonseJeonyongUnitPriceAverage'],
         df.iloc[0]['buildingAreaPerLandArea'],
         df.iloc[0]['floorAreaRatio'],
         df.iloc[0]['mainPurposeCodeName'],
         df.iloc[0]['otherPurpose'],
         df.iloc[0]['saedaeCount'],
         df.iloc[0]['totalParkingLotCount'],
         df.iloc[0]['distance']
        ],
    tooltip = danjiName,
    icon = folium.Icon('red', icon = icon[0])).add_to(m)
 
    
for i in range(0, len(train)):
    folium.Marker(
        location = [train.iloc[i]['lat'], train.iloc[i]['lon']],
        popup = train.iloc[i]['lineName'],
        tooltip = train.iloc[i]['lineName'],
        icon = folium.Icon('green', icon = icon[1])).add_to(m)
m    


### pydeck 활용

In [175]:
import pydeck as pdk

In [176]:
from shapely import wkt
df['coord'] = df['coord'].apply(wkt.loads)

In [177]:
for i in range(0, len(df)):
    x, y  = df['coord'][i].xy
    df['coord'][i] = [[x[0], y[0]]]

C:\Users\JEONGS~1\AppData\Local\Temp/ipykernel_8688/2738120909.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['coord'][i] = [[x[0], y[0]]]


In [178]:
df['coord']

0      [[127.018035042288, 37.5150660593812]]
1      [[127.015897586489, 37.5134134427106]]
2       [[127.01728838332, 37.5127589883983]]
3       [[127.01728838332, 37.5127589883983]]
4       [[127.01728838332, 37.5127589883983]]
                        ...                  
305    [[127.020924508318, 37.5235708121572]]
306    [[127.020924508318, 37.5235708121572]]
307    [[127.023212732073, 37.5229678022058]]
308    [[127.023212732073, 37.5229678022058]]
309    [[127.023212732073, 37.5229678022058]]
Name: coord, Length: 310, dtype: object

In [179]:
df

,pnuCode,addressJibeon,addressRoad,danjiName,usePermissionDate,contractYear,contractMonth,jeonyongArea,jeonseJeonyongUnitPriceAverage,buildingAreaPerLandArea,floorAreaRatio,mainPurposeCodeName,otherPurpose,saedaeCount,totalParkingLotCount,lon,lat,coord,geom,distance
0,1165010600100190004,서울특별시 서초구 잠원동 19-4번지,서울특별시 서초구 강남대로99길 28,새서울맨션,,2021,6,148.30,2229.120111,0,0,공동주택,공동주택,20,0,127.018035,37.515066,"[[127.018035042288, 37.5150660593812]]","b""\x00\x00\x00\x00\x01\x01\x00\x00\x00\xc63s|'...",2.477691e-08
1,1165010600100170010,서울특별시 서초구 잠원동 17-10번지,서울특별시 서초구 신반포로45길 88,유로미,20050906,2021,6,84.67,1776.464192,51,226,공동주택,공동주택(아파트11세대)),11,12,127.015898,37.513413,"[[127.015897586489, 37.5134134427106]]",b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\x8f\x83...,2.632671e+02
2,1165010600100310014,서울특별시 서초구 잠원동 31-14번지,서울특별시 서초구 강남대로95길 56,대주피오레아파트,20021231,2021,12,84.98,2409.901017,35,236,공동주택,공동주택(아파트),49,78,127.017288,37.512759,"[[127.01728838332, 37.5127589883983]]",b'\x00\x00\x00\x00\x01\x01\x00\x00\x00r=\xbc@\...,2.648521e+02
3,1165010600100310014,서울특별시 서초구 잠원동 31-14번지,서울특별시 서초구 강남대로95길 56,대주피오레아파트,20021231,2020,9,117.93,2354.667603,35,236,공동주택,공동주택(아파트),49,78,127.017288,37.512759,"[[127.01728838332, 37.5127589883983]]",b'\x00\x00\x00\x00\x01\x01\x00\x00\x00r=\xbc@\...,2.648521e+02
4,1165010600100310014,서울특별시 서초구 잠원동 31-14번지,서울특별시 서초구 강남대로95길 56,대주피오레아파트,20021231,2020,6,121.83,2252.155998,35,236,공동주택,공동주택(아파트),49,78,127.017288,37.512759,"[[127.01728838332, 37.5127589883983]]",b'\x00\x00\x00\x00\x01\x01\x00\x00\x00r=\xbc@\...,2.648521e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,1168011000103970000,서울특별시 강남구 압구정동 397번지,서울특별시 강남구 압구정로 113,미성2차,,2022,4,118.63,2925.966771,0,0,공동주택,주거시설,910,0,127.020925,37.523571,"[[127.020924508318, 37.5235708121572]]",b'\x00\x00\x00\x00\x01\x01\x00\x00\x00I\xba\xb...,9.794166e+02
306,1168011000103970000,서울특별시 강남구 압구정동 397번지,서울특별시 강남구 압구정로 113,미성2차,,2022,3,140.90,2627.735514,0,0,공동주택,주거시설,910,0,127.020925,37.523571,"[[127.020924508318, 37.5235708121572]]",b'\x00\x00\x00\x00\x01\x01\x00\x00\x00I\xba\xb...,9.794166e+02
307,1168010700105470008,서울특별시 강남구 신사동 547-8번지,서울특별시 강남구 압구정로14길 20,유로미아파트B동,20040910,2018,3,102.01,2268.453668,42,257,공동주택,"공동주택 (아파트), 근린생활시설",19,32,127.023213,37.522968,"[[127.023212732073, 37.5229678022058]]",b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\xaeFAQ|...,9.902100e+02
308,1168010700105470008,서울특별시 강남구 신사동 547-8번지,서울특별시 강남구 압구정로14길 20,유로미아파트B동,20040910,2022,2,116.86,2913.707580,42,257,공동주택,"공동주택 (아파트), 근린생활시설",19,32,127.023213,37.522968,"[[127.023212732073, 37.5229678022058]]",b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\xaeFAQ|...,9.902100e+02


In [180]:
df = df.drop(columns = 'geom')

In [181]:
df = pd.read_json(df.to_json())

In [182]:
df

,pnuCode,addressJibeon,addressRoad,danjiName,usePermissionDate,contractYear,contractMonth,jeonyongArea,jeonseJeonyongUnitPriceAverage,buildingAreaPerLandArea,floorAreaRatio,mainPurposeCodeName,otherPurpose,saedaeCount,totalParkingLotCount,lon,lat,coord,distance
0,1165010600100189952,서울특별시 서초구 잠원동 19-4번지,서울특별시 서초구 강남대로99길 28,새서울맨션,,2021,6,148.30,2229.120111,0,0,공동주택,공동주택,20,0,127.018035,37.515066,"[[127.0180350423, 37.5150660594]]",2.480000e-08
1,1165010600100169984,서울특별시 서초구 잠원동 17-10번지,서울특별시 서초구 신반포로45길 88,유로미,20050906,2021,6,84.67,1776.464192,51,226,공동주택,공동주택(아파트11세대)),11,12,127.015898,37.513413,"[[127.0158975865, 37.5134134427]]",2.632671e+02
2,1165010600100310016,서울특별시 서초구 잠원동 31-14번지,서울특별시 서초구 강남대로95길 56,대주피오레아파트,20021231,2021,12,84.98,2409.901017,35,236,공동주택,공동주택(아파트),49,78,127.017288,37.512759,"[[127.0172883833, 37.5127589884]]",2.648521e+02
3,1165010600100310016,서울특별시 서초구 잠원동 31-14번지,서울특별시 서초구 강남대로95길 56,대주피오레아파트,20021231,2020,9,117.93,2354.667603,35,236,공동주택,공동주택(아파트),49,78,127.017288,37.512759,"[[127.0172883833, 37.5127589884]]",2.648521e+02
4,1165010600100310016,서울특별시 서초구 잠원동 31-14번지,서울특별시 서초구 강남대로95길 56,대주피오레아파트,20021231,2020,6,121.83,2252.155998,35,236,공동주택,공동주택(아파트),49,78,127.017288,37.512759,"[[127.0172883833, 37.5127589884]]",2.648521e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,1168011000103970048,서울특별시 강남구 압구정동 397번지,서울특별시 강남구 압구정로 113,미성2차,,2022,4,118.63,2925.966771,0,0,공동주택,주거시설,910,0,127.020925,37.523571,"[[127.0209245083, 37.5235708122]]",9.794166e+02
306,1168011000103970048,서울특별시 강남구 압구정동 397번지,서울특별시 강남구 압구정로 113,미성2차,,2022,3,140.90,2627.735514,0,0,공동주택,주거시설,910,0,127.020925,37.523571,"[[127.0209245083, 37.5235708122]]",9.794166e+02
307,1168010700105469952,서울특별시 강남구 신사동 547-8번지,서울특별시 강남구 압구정로14길 20,유로미아파트B동,20040910,2018,3,102.01,2268.453668,42,257,공동주택,"공동주택 (아파트), 근린생활시설",19,32,127.023213,37.522968,"[[127.0232127321, 37.5229678022]]",9.902100e+02
308,1168010700105469952,서울특별시 강남구 신사동 547-8번지,서울특별시 강남구 압구정로14길 20,유로미아파트B동,20040910,2022,2,116.86,2913.707580,42,257,공동주택,"공동주택 (아파트), 근린생활시설",19,32,127.023213,37.522968,"[[127.0232127321, 37.5229678022]]",9.902100e+02


In [189]:
layer = pdk.Layer(
    'ScatterplotLayer',
    df,
    get_position='[lng, lat]',
    get_radius=50,
    get_fill_color='[255, 255, 255]',
    pickable=True,
    auto_highlight=True
)

37.515066059381176

In [193]:
# Make layer
color  = np.random.choice(range(256), size=3)


layer = pdk.Layer(
    'ScatterplotLayer', # 사용할 Layer 타입
    df, # 시각화에 쓰일 데이터프레임
    get_position='[lon, lat]',
    get_radius = 50,
    get_fill_color='[255, 255, 255]',# 각 데이터 별 rgb 또는 rgba 값 (0~255)
    pickable=True, # 지도와 interactive 한 동작 on
    auto_highlight=True # 마우스 오버(hover) 시 박스 출력
)

# Set the viewport location
center = [center[0], center[1]]
view_state = pdk.ViewState(
    longitude=center[0],
    latitude=center[1],
    zoom=10)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

In [197]:
geo

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[127.00864326221883, 37.58046825113493],
       [127.00871274905404, 37.58045116421941],
       [127.00876564011087, 37.58044310616639],
       [127.00890785297045, 37.58042423069648],
       [127.00913781377906, 37.58039352848355],
       [127.00916523299747, 37.580390715633335],
       [127.00923792440939, 37.58038254796624],
       [127.00926870590294, 37.58037916725067],
       [127.0092779097711, 37.580378031403306],
       [127.0094381678049, 37.58035663134507],
       [127.0100319799304, 37.58027803301481],
       [127.01003587429, 37.58027746511249],
       [127.0102959050288, 37.580266183737045],
       [127.01059113253234, 37.580253772856366],
       [127.01067835517419, 37.58039257208789],
       [127.01070383209942, 37.58043030341881],
       [127.01097470383016, 37.58082728365548],
       [127.01111447910849, 37.58104406770486],
       [127.01117197